In [1]:
import gc
import os
import re
import string

import en_core_web_lg
import numpy as np
import pandas as pd
import spacy
import utils_clean
from nltk.corpus import stopwords
from tqdm import tqdm

In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train.fillna('CVxTz', inplace = True)
test.fillna('CVxTz', inplace = True)

test

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
5,0001ea8717f6de06,Thank you for understanding. I think very high...
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...
7,000247e83dcc1211,:Dear god this site is horrible.
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ..."
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...


In [3]:
train2 = utils_clean.apply_wordlist_clean(train)
train2 = utils_clean.apply_basic_cleaning(train2)

test2 = utils_clean.apply_wordlist_clean(test)
test2 = utils_clean.apply_basic_cleaning(test2)

  0%|          | 0/1 [00:00<?, ?it/s]

Glove preprocessing.
Text to wordlist preprocessing.


  0%|          | 0/1 [00:00<?, ?it/s]

Glove preprocessing.
Text to wordlist preprocessing.


100%|██████████| 1/1 [00:06<00:00,  6.47s/it]


In [5]:
train2.to_pickle('../data/train_basic_clean2.pkl')
test2.to_pickle('../data/test_basic_clean2.pkl')

In [ ]:
train2 = pd.read_pickle('../data/train_basic_clean.pkl')
test2 = pd.read_pickle('../data/test_basic_clean.pkl')

train2['id'] = train.id
test2['id'] = test.id

train2.to_pickle('../data/train_basic_clean.pkl')
test2.to_pickle('../data/test_basic_clean.pkl')

In [ ]:
nlp = spacy.load('en')
stops = stopwords.words("english")

q1 = []
for doc in tqdm(nlp.pipe(train['comment_text'], n_threads=8, batch_size=128)):
    q1.append(utils_clean.normalize_spacy(doc))


q2 = []
for doc in tqdm(nlp.pipe(test['comment_text'], n_threads=8, batch_size=128)):
    q2.append(utils_clean.normalize_spacy(doc))

In [ ]:
q1 = pd.Series(q1)
q2 = pd.Series(q2)

train.comment_text = q1
test.comment_text = q2

train.to_pickle('../data/train_spacy_clean.pkl')
test.to_pickle('../data/test_spacy_clean.pkl')

train